In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# The probably most difficult Sudoku

Nearly everyone know Sudokus. You can find them in newspapers, puzzle magazines, apps and many more media. They are a popular pastime. The task is to fill out a $9 \times 9$ square such that every number from $1 - 9$ appears only once in each row, each column and each of the $9$ $3 \times 3$ blocks. If you need a refresher of the rules or if you are not familiar with Sudokus, please have a look at the [Wikipedia article](https://en.wikipedia.org/wiki/Sudoku).

There are Sudokus for all levels: from beginner to expert. Some of them seem extremely difficult and we might think that only few people are able to solve them. You can find a variety of hard Sudokus on the following websites:
* [AISudokus Top 10](http://www.aisudoku.com/en/AIsudoku_Top10s1_en.pdf)
* [Forum Sudoku Dragon](https://www.sudokudragon.com/forum/hardest-puzzles.htm)
* [OE24 - Das härteste Sudoku der Welt (German)](https://www.oe24.at/welt/das-ist-das-schwierigste-sudoku-der-welt/1597831)

Now, we present probably the hardest Sudoku known:

<img src="./figures/hardest_input.png" width="400"/>

In the following, a Sudoku game which you can play in the Jupyter notebook is implemented. It shows the Sudoku above and you are invited to solve it.

## Try to solve this on your own

By executing the next cell, a Sudoku game will be created. We already integrated the Sudoku above into it, so you can try to solve this Sudoku on your own. The game will give you some hints, i.e., forbidden numbers for cells.

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import numpy as np
import json
import math
import os

font_size = 24
display(widgets.HTML('<style>.grey_cell select { background-color:#D3D3D3; font-size:' + str(font_size) + 'px; font-weight: bold; !important; }</style>'))
display(widgets.HTML('<style>.ivory_cell select { background-color:#FFFFF0; font-size:' + str(font_size) + 'px; font-weight: bold; !important; }</style>'))

SIZE = 9
sqrtSIZE = int(math.sqrt(SIZE))
        
class Board:

    def __init__(self, filename:str=None):
        
        if (filename is None) or not (os.path.isfile(filename)):
            self.game = [[0 for column in range(SIZE)] for row in range(SIZE)]
            self._fixed = [[False for column in range(SIZE)] for row in range(SIZE)]
        else:
            with open(filename) as filehandle:
                self.game = json.load(filehandle)
                self._fixed = [[self.game[row][column] > 0 for column in range(SIZE)] for row in range(SIZE)]
                
        self.info = widgets.Label('')

    def set_value(self, row, column, value):
        
        if (row in range(SIZE)) and (column in range(SIZE)) and (value in range(SIZE)):
            self.game[row][column] = value + 1

    def get_constant_bits(self):

        constant_bits = np.full((SIZE, SIZE, SIZE), -1, np.int8)

        def set_cell_value(row, column, value):
            for h_value in range(SIZE):
                if h_value != value:
                    if constant_bits[row, column, h_value] == 1:
                        raise ValueError('Invalid Sudoku!')
                    constant_bits[row, column, h_value] = 0

        def set_row_value(row, column, value):
            for h_row in range(SIZE):
                if h_row != row:
                    if constant_bits[h_row, column, value] == 1:
                        raise ValueError('Invalid Sudoku!')
                    constant_bits[h_row, column, value] = 0

        def set_column_value(row, column, value):
            for h_column in range(SIZE):
                if h_column != column:
                    if constant_bits[row, h_column, value] == 1:
                        raise ValueError('Invalid Sudoku!')
                    constant_bits[row, h_column, value] = 0

        def set_block_value(row, column, value):
            grid_row = row // sqrtSIZE
            grid_column = column // sqrtSIZE
            for h_row in range(grid_row * sqrtSIZE, grid_row * sqrtSIZE + sqrtSIZE):
                for h_column in range(grid_column * sqrtSIZE, grid_column * sqrtSIZE + sqrtSIZE):
                    if (h_row != row) and (h_column != column):
                        if constant_bits[h_row, h_column, value] == 1:
                            raise ValueError('Invalid Sudoku!')
                        constant_bits[h_row, h_column, value] = 0

        for row in range(SIZE):
            for column in range(SIZE):
                if self.game[row][column] > 0:
                    value = self.game[row][column] - 1
                    set_cell_value(row, column, value);
                    set_row_value(row, column, value);
                    set_column_value(row, column, value);
                    set_block_value(row, column, value);
                    constant_bits[row, column, value] = +1
        return constant_bits

    def show(self):
        self._display(disabled=True)
        
    def edit(self):
        self._display(disabled=False)
    
    def _display(self, disabled:bool = False):
        self._update = False
        bits = self.get_constant_bits()
        cells = [[widgets.Dropdown(value='', options=[''],
                                   disabled=disabled or self._fixed[row][column], 
                                   layout=widgets.Layout(width=f'{font_size+22}px', height=f'{font_size+22}px')) for column in range(SIZE) ]for row in range(SIZE)]

        if not disabled:
            def handle_change(change):
                if not self._update:
                    return
                self._update = False
                self.info.value = ''
                row, column = change['owner'].position
                self.game[row][column] = change['new'] if (change['new'] != '') else 0
                try:
                    bits = self.get_constant_bits()
                    for row in range(SIZE):
                        for column in range(SIZE):
                            cells[row][column].options = [''] + [(value+1) for value in range(SIZE) if bits[row, column, value] != 0]
                            cells[row][column].value = self.game[row][column] if (self.game[row][column] != 0) else ''
                except ValueError as oops:
                    self.info.value = str(oops)
                self._update = True

        for row in range(SIZE):
            for column in range(SIZE):
                cells[row][column].options = [''] + [(value+1) for value in range(SIZE) if bits[row, column, value] != 0]
                cells[row][column].value = self.game[row][column] if (self.game[row][column] != 0) else ''
                cells[row][column].position = (row, column)
                if not disabled:
                    cells[row][column].observe(handle_change, names='value')
                cells[row][column].add_class('grey_cell' if (row//sqrtSIZE + column//sqrtSIZE) % 2 == 1 else 'ivory_cell')      

        grid_row_widgets = []
        for grid_row  in range(sqrtSIZE):
            grid_column_widgets = []
            for grid_column in range(sqrtSIZE):
                row_widgets = []
                for cell_row in range(sqrtSIZE):
                    row_widgets.append(widgets.HBox(cells[(grid_row * sqrtSIZE + cell_row)][grid_column * sqrtSIZE: (grid_column+1) * sqrtSIZE]))    
                grid_column_widgets.append(widgets.VBox(row_widgets))
            grid_row_widgets.append(widgets.HBox(grid_column_widgets))
        board_widget =widgets.VBox(grid_row_widgets)

        display(widgets.VBox([board_widget, self.info]))
        self._update = True
        
        try:
            self.get_constant_bits()
        except ValueError as oops:
            self.info.value = str(oops)          

print("*-*--*")
print("Sudoku")
print("*--*--*")
print()
print("The grey numbers are the given one, so you can't change them.")
print("Try to fill in the missing numbers by clicking on the little arrows and choosing a number.")

print()
Board('examples/hardest.json').edit()

# Sudokus as Combinatorial Optimization Problems

Computers can solve Sudokus similar to human. Following the rules of uniqueness for rows, columns and sub-squares for every unknown field certain solutions can be excluded. If this brings down the number of candidates to a single choice, the field can be entered and with the new information the exclusion for the remaining open fields can be refined. If more than one candidate is possible for all fields, then two or more hypothetical solutions have to be tested and all but one will lead to a contradiction. Single or few of such ambiguous points can be handled by humans. In the example above there are 10 such choices with two alternatives, hence we get an hypothesis tree with 11 leaves. It is hard for a human player to test all those systematically. For a computer program the effort is not harder than solving 11 simple Sudokus and it is possible in the fraction of a second. So in the case of Sudoku using Annealing is not the last resort to solve a problem that cannot be solved otherwise. Nevertheless the combinatorial Sudoku rules can nicely be translated into a QUBO model and for that reason we hope you enjoy the instructive example though there are more effective means to solve the Sudoku problem classically.

## Definitions

Have you solved the Sudoku? If not, don't be disappointed, it's one of the most difficult ones.

Next, we set the stage to solve Sudokus using Quadratic Unconstraint Optimization problems. This means we have to formulate the Sudoku rules in form of a QUBO model. To this end, we define the following variables:
* $N=[1,\dots,9]$
* The $9$ rows are denoted by $r\in N$ (In the figures indicated as numbers in blue circles).
* The $9$ columns are denoted by $c\in N$ (In the figures indicated as numbers in a red circle).
* The $9$ values are denoted by $v\in N$.
* The $9$ blocks are denoted by $b\in N$ (In the figures indicated as ivory and gray blocks). I.e., block $1$ includes the $9$ cells ranging from $r=1,\dots,3$ and $c=1,\dots,3$.
* Each cell represented by the tuple $(r,c)$ contains exactly one value $v$.

We define the decision variable $x$ as:
$$
x_{r, c, v} = \begin{cases}
    1 & \text{if cell } (r, c) \text{ contains value } v \\ 
    0 & \text{otherwise.}
\end{cases}
$$
We can visualize the space of the decision variable as shown in the following Figure.

![Overview](./figures/bits-overview.png)

In the following Figure the cell in the upper right corner, represented by the tuple $(9,9)$, is highlighted (red). 

![Special Bit](./figures/bits-special.png)

If for example the value of the cell $(9,9)$ is $v = 7$, then
$$
x_{r, c, v} = x_{9, 9, 7} = 1 \, .
$$

We can also calculate how many bits are needed, to solve a Sudoku. Each cell $(r,c)$ contains one of $9$ numbers. For each number we have to set $x=0$ or $x=1$. The total number of bits is $N_{bits} = 9^2 \text{ cells }\times 9 \text{ values }=9^3=792$.

## Building the QUBO

The QUBO is setup "rule by rule". In each step we formulate one of the Sudoku rules as a QUBO.

In the case of solving a Sudoku there is no optimization objective as you might have encountered in the other examples. Here, it is all about obeying the Sudoku rules. Therefore, the QUBO only contains "penalty"-terms. The goal in "optimization" of such a QUBO is to find a valid solution, i.e. a solution with all penalties equal to zero.

### First Step: Each cell has exactly one value

We start with the first rule: Each cell must contain exactly one value. This is obvious for us, but of course not for the Digital Annealer:

As an example, we show in the following Figure the case for cell $(r=9, c=9)$ to better understand the rule.
![Figure Cell A](./figures/cell/A.gif)
In the cell $(r=9, c=9)$ we have to place exactly one value $v$. The penalty for this example reads:
$$
p  = \left( \sum _{v \, \in \, N} x_{9, 9, v} - 1 \right) ^{2}
$$
If one of the terms in the sum $\sum _{v \, \in \, N}$ is one, then the penalty is zero. Otherwise we violate the rule and pay a penalty greater zero.

<br></br>
Of course this rule applies to all cells, as shown in the Figure below.
![Figure Cell B](./figures/cell/B.gif)
The corresponding QUBO formulation reads: 
$$
P_{cell} = \sum_{r \, \in \, N} \sum_{c \, \in \, N} \left( \sum _{v \, \in \, N} x_{r, c, v} - 1 \right) ^{2}
$$

### Second Step: Each value to appear exactly once in each row

Each value $v = 1,\dots,9$ is allowed to appear only once in each row $r$.

As an example, we show in the following Figure the case $r = 1$ and $v = 1$ to better understand the rule.
![Figure Row A](./figures/row/A.gif)
If we want to place $v = 1$ in row $r=1$ we can do this in one of the columns. The penalty for placing the value in more than one column in row $r=1$ in this example reads:
$$
p = \left( \sum _{c \, \in \, N} x_{1, c, 1} - 1 \right) ^{2}
$$
If one of the terms in the sum $\sum _{c \, \in \, N}$ is one, then the penalty is zero. Otherwise we violate the rule and pay a penalty greater zero.

<br></br>
Of course this rule applies to all values in all rows, as shown in the Figure below.
![Figure Row B](./figures/row/B.gif)
The corresponding QUBO formulation reads: 
$$
P_{row} = \sum_{v \, \in \, N} \sum_{r \, \in \, N} \left( \sum _{c \, \in \,N} x_{r, c, v} - 1 \right) ^{2}
$$

### Third Step: Each value to appear exactly once in each column

This rule is similar to the one before. Each value $v$ is allowed to appear only once in each column $c$.

As an example, we show in the following Figure the case $c = 1$ and $v = 1$ to better understand the rule.
![Figure Column A](./figures/column/A.gif)
If we want to place $v = 1$ in column $c=1$ we can do this in one of the rows. The penalty for placing the value in more than one row in column $c=1$ in this example reads:
$$
p = \left( \sum _{r \, \in \, N} x_{r, 1, 1} - 1 \right) ^{2}
$$

<br></br>
Of course this rule applies to all values in all columns, as shown in the Figure below.
![Figure Column B](./figures/column/B.gif)
The corresponding QUBO formulation reads: 
$$
P_{column} = \sum_{v \, \in \, N} \sum_{c \, \in \, N} \left( \sum _{r \, \in \, N} x_{r, c, v} - 1 \right) ^{2}
$$

### Fourth Step: Each value to appear exactly once in each block

Now we deal with the rule concerning the $9$ blocks. As before each value $v$ is allowed to appear only once in each block $b$.

As an example, we show in the following Figure the case $b = 1$ and $v = 1$ to better understand the rule.
![Figure Block A](./figures/block/A.gif)
If we want to place $v = 1$ in block $b=1$ we can do this in one of the three rows and three columns of this block. The penalty for placing the value more than one in block $b=1$ in this example reads:
$$
p = \left( \sum _{(r, c) \, \in \, b=1} x_{r, c, 1} - 1 \right) ^{2}
$$

<br></br>
Of course this rule applies to all values in all blocks, as shown in the Figure below.
![Figure Block B](./figures/block/B.gif)
The corresponding QUBO formulation reads: 
$$
P_{block} = \sum_{v \, \in \, N} \sum_{b \, \in \, N} \left( \sum _{(r, c) \, \in \, b} x_{r, c, v} - 1 \right) ^{2}
$$

### Fifth Step: Reducing the solution complexity

We can reduce the complexity of the problem with the following considerations. If value $v$ is set in a specific cell, some possible solutions can be excluded.
For instance, given value $v$ in cell $z$ (green cell), i.e. the corresponding decision variable $x_{r,c,v}$ is set to $1$, all decision variables $x_{r,c,v}$ corresponding to the blue cells must be set to $0$. This example is depicted in the Figure below:
![Figure Hint](./figures/hint/a.gif)

To summarize there must not be
* another value in the same cell
* another cell with same value in same row $r$
* another cell with same value in same column $c$
* another cell with same value in same block $b$

### The total QUBO P

Finally, we can setup the total QUBO for solving a Sudoku according to the rules:

$$
P = P_{cell} + P_{row} + P_{column} + P_{block}
$$

## Solving the Sudoku with the Digital Annealer

Now it is time to solve the Sudoku with the Digital Annealer. The next cell contains an implementation of the QUBO formulation presented above.

Please execute the next cell. This will show the `Optimizer` where you can step by step go through the ***Setup scenario***, ***Build QUBO***, ***Solve annealing***, ***Annealer tracker*** and ***Report*** tabs.

1. Under the ***Setup scenario*** tab please choose the Sudoku you want to be solved. If you do this for the first time, choose `Mode: FILE` and `File: hardest.json` from the example folder. Then press `Call`.

2. After the Sudoku is loaded, go the the tab ***Build QUBO*** and press `Call`. Now the QUBO is build.

3. Select the ***Solve annealing*** tab and press `Call`. In this tab you can also adjust some annealing specific parameters, but that is not necessary for the first try. If you use the CPU based emulator, it can happen that the solution is not valid. Do not worry about it, just call the solver again until you have a valid solution.

4. After the solver produced a solution, you can find more information on the different timings under the ***Annealer tracker*** tab.

5. The ***Report*** tab provides information on the bit configuration of the solution.

In [ ]:
from dadk.Optimizer import *

class SudokuModel(OptimizerModel):

    def __init__(self, persistent_file="Sudoku.dao"):

        super().__init__(
            name='Sudoku',
            persistent_file=persistent_file,
            load_parameter=[
                {'name': 'mode', 'value': 'FILE', 'type': 'select', 'options':['EDIT', 'FILE'],
                    'label': 'Mode', 'description': 'Edit board or load from file.',
                    'on_change': 'filename.visible = new_value == "FILE"'},
                {'name': 'filename', 'value': '', 'type': 'file_selector', 'suffix': 'json',
                    'label': 'File', 'description': 'File with suffix json for board definition'}
            ])

    # ----------------------------------------------------------------------------------------------------

    def load(self, mode:str, filename:str, silent=False):

        if mode == 'FILE':
            self.board = Board(filename)
            self.board.show()
        else:
            self.board = Board()
            self.board.edit()            

    # ----------------------------------------------------------------------------------------------------

    def build_qubo(self, A=100, silent=False):
        
        BinPol.freeze_var_shape_set(VarShapeSet(BitArrayShape('x', (SIZE, SIZE, SIZE),
                                                  constant_bits=self.board.get_constant_bits(),
                                                  one_hot=OneHot.one_way,
                                                  axis_names=["Row", "Column", "Value"])))
        
        self.P_cell = BinPol()
        for r in range(SIZE):
            for c in range(SIZE):
                self.P_cell += (BinPol.sum(Term(1, ('x', r, c, v)) for v in range(SIZE)) - 1) ** 2

        self.P_row = BinPol()
        for v in range(SIZE):
            for r in range(SIZE):
                self.P_row += (BinPol.sum(Term(1, ('x', r, c, v)) for c in range(SIZE)) - 1) ** 2

        self.P_column = BinPol()
        for v in range(SIZE):
            for c in range(SIZE):
                self.P_column += (BinPol.sum(Term(1, ('x', r, c, v)) for r in range(SIZE)) - 1) ** 2

        self.P_block = BinPol()
        for v in range(SIZE):
            for grid_r in range(sqrtSIZE):
                for grid_c in range(sqrtSIZE):
                    self.P_block += (BinPol.sum(Term(1, ('x', r, c, v))
                                                for r in range(grid_r * sqrtSIZE, grid_r * sqrtSIZE + sqrtSIZE)
                                                for c in range(grid_c * sqrtSIZE, grid_c * sqrtSIZE + sqrtSIZE)) - 1) ** 2

        self.HQ = A * (self.P_cell + self.P_row + self.P_column + self.P_block)

        if not silent:
            print('QUBO with %d bits created.' % self.HQ.N)

    # ----------------------------------------------------------------------------------------------------

    def prep_result(self, solution_list: SolutionList, silent=False):

        self.solution = solution_list.min_solution

        if not silent:
            print("HQ       = %8.2f" % self.HQ.compute(self.solution.configuration))
            print("P_cell   = %8.2f" % self.P_cell.compute(self.solution.configuration))
            print("P_row    = %8.2f" % self.P_row.compute(self.solution.configuration))
            print("P_column = %8.2f" % self.P_column.compute(self.solution.configuration))
            print("P_block  = %8.2f" % self.P_block.compute(self.solution.configuration))

            
        if self.P_cell.compute(self.solution.configuration) == 0 and self.P_row.compute(self.solution.configuration) == 0 and self.P_column.compute(self.solution.configuration) == 0 and self.P_block.compute(self.solution.configuration) == 0:        
            solution_board = Board()
            for (row, column, value) in np.argwhere(self.solution['x'].data == 1):
                solution_board.set_value(row, column, value)
            solution_board.show()
        else:
            print('Invalid solution! Try again!')

    # ----------------------------------------------------------------------------------------------------

    def report(self, silent=False):

        self.solution['x'].draw(figsize=(12.0, 13.0), max_ax_per_row=3, order=[1, 0, 2], axis_offsets=[1, 1, 1])


optimizer = Optimizer(SudokuModel(persistent_file='Sudoku.dao'))

## See How Fast the Digital Annealer Solves the Sudoku

In the following cell, we load a solution (``Sudoku_solution_DA.dao``) to the hardest Sudoku (``hardest.json``) which was obtained using the Digital Annealer in just **0.779 sec**.

In [ ]:
Optimizer.select_instance(
    optimizer_model=SudokuModel,
    dao_path='.',
    initial_selection="Sudoku_solution_DA.dao", immediate_load=True, read_only=True
)

## Solution

If you were not able to find a valid solution with the annealing process by the emulator you can find the solution right here:
<img src="./figures/hardest_solve.png" width="400"/>

## Try out more

You can also setup your own Sudokus in the `Optimizer` choosing `Mode: Edit`. We wish you a good time creating different Sudokus and trying to beat the Digital Annealer!